In [126]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier 

import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to /home/csuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
data = pd.read_csv('superheroes_nlp_dataset.csv')
data.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
data = data.drop(data.columns.drop('creator').drop('history_text'), axis=1)
data.dropna(inplace=True)

In [129]:
def get_text_processing(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

In [130]:
data['history_text'] = data['history_text'].apply(get_text_processing)
data

,history_text,creator
0,Delroy Garrett Jr grew become track star compe...,Marvel Comics
1,one many prisoners Indian Hill transferred ano...,DC Comics
2,Richard Rick Jones orphaned young age expelled...,Marvel Comics
3,Aa one passive members Pumice People race Ston...,DC Comics
4,Aaron Cash head security Arkham Asylum hook ha...,DC Comics
5,ayla Secura Rutian Twilek Jedi Knight onetime ...,George Lucas
6,Sapien began life Langdon Everett Caul Victori...,Dark Horse Comics
7,Originally history professor planet Ungara Abi...,DC Comics
8,Formerly known Emil Blonsky spy Soviet Yugosla...,Marvel Comics
9,Abra Kadabra criminal time traveler 64th centu...,DC Comics


In [77]:

X = data['history_text'].values
Y = data['creator'].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [79]:
cv = CountVectorizer()
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)

In [80]:
tfidf = TfidfTransformer()
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

x_train = x_train.toarray()
x_test = x_test.toarray()

In [81]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [82]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=200)
rfc.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1)

In [83]:
rfc.score(x_test, y_test)

0.7351351351351352

In [131]:
one_hot = pd.get_dummies(data['creator'])
data.drop(['creator'],axis=1,inplace=True)
data = pd.concat([data,one_hot],axis=1)

In [132]:
X = data['history_text'].values
Y = data.drop('history_text', axis=1).values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

cv = CountVectorizer()
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)

tfidf = TfidfTransformer()
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

x_train = x_train.toarray()
x_test = x_test.toarray()

In [133]:
model = Sequential()

model.add(Dense(units=12673,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=4000,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=500,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=3, activation='softmax'))

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [137]:
model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), verbose=1, callbacks=early_stop)

TypeError: 'NoneType' object is not callable

In [125]:
x_train

array([[-0.03406014, -0.03406014, -0.03406014, ..., -0.04734469,
        -0.03406014, -0.04572589],
       [-0.03406014, -0.03406014, -0.03406014, ..., -0.04734469,
        -0.03406014, -0.04572589],
       [-0.03406014, -0.03406014, -0.03406014, ..., -0.04734469,
        -0.03406014, -0.04572589],
       ...,
       [-0.03406014, -0.03406014, -0.03406014, ..., -0.04734469,
        -0.03406014, -0.04572589],
       [-0.03406014, -0.03406014, -0.03406014, ..., -0.04734469,
        -0.03406014, -0.04572589],
       [-0.03406014, -0.03406014, -0.03406014, ..., -0.04734469,
        -0.03406014, -0.04572589]])